In [240]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py 
import plotly.graph_objs as go 

def del_NONE_index(df):
    j=[]
    for i in list(df.index):
        if df.loc[i,'player1_lane']=='NONE':
            j.append(i)
    df.drop(j,inplace=True)
    
def plot_bar_horizontal(col_name: str, target: str, title_name: str):
    cross_table = round(pd.crosstab(df[col_name], df[target], normalize='index')*100, 2)
    print(df[col_name].value_counts())
    index0 = cross_table.columns.tolist()[0] 
    index1 = cross_table.columns.tolist()[1] 
    trace0 = go.Bar(x=cross_table[index0].values.tolist(),
                    y=cross_table.index.tolist(), 
                    name=index0,
                    orientation='h',
                    marker=dict(color='rgb(0,255,255)')
                   ) 
    trace1 = go.Bar(x=cross_table[index1].values.tolist(),  
                    y=cross_table.index.tolist(),
                    name=index1,
                    orientation='h',
                    marker=dict(color='rgb(255,0,255)')
                   ) 

    data = [trace0, trace1] 
    layout = go.Layout(title=title_name, bargap=0.4, barmode='stack')
    fig = go.Figure(data=data, layout=layout)
    chart=title_name+'.html'
    py.offline.plot(fig,filename=chart)

def team1_win():
    trace0 = go.Pie(labels=df['team1_win'].value_counts().index, 
            values=df['team1_win'].value_counts().values,
            hole=0.5,
            marker=dict(line=dict(color='purple'))
           ) 
    layout = go.Layout(title='team1胜率统计')
    data = [trace0]
    fig = go.Figure(data, layout)
    py.offline.plot(fig,filename='team1胜率统计.html')
    
def team_gold():
    raw1=df.loc[:,['player1_goldEarned','player2_goldEarned','player3_goldEarned','player4_goldEarned','player5_goldEarned']]
    raw2=df.loc[:,['player6_goldEarned','player7_goldEarned','player8_goldEarned','player9_goldEarned','player10_goldEarned']]
    win=df.loc[:,'team1_win']
    raw1['gold1'] = raw1.apply(lambda x: x.sum(), axis=1)
    raw2['gold2'] = raw2.apply(lambda x: x.sum(), axis=1)
    gold=pd.concat([raw1['gold1'],raw2['gold2'],win],axis=1)
    gold['diff'] = gold['gold1']-gold['gold2']
    return gold

def team_KDA(df):
    raw1=df.iloc[:,12:43]
    raw2=df.iloc[:,44:78]
    win=df.loc[:,'team1_win']
    raw1['t1_kills']=raw1['player1_kills']+raw1['player2_kills']+raw1['player3_kills']+raw1['player4_kills']+raw1['player5_kills']
    raw1['t1_deaths']=raw1['player1_deaths']+raw1['player2_deaths']+raw1['player3_deaths']+raw1['player4_deaths']+raw1['player5_deaths']
    raw1['t1_assists']=raw1['player1_assists']+raw1['player2_assists']+raw1['player3_assists']+raw1['player4_assists']+raw1['player5_assists']
    raw2['t2_kills']=raw2['player6_kills']+raw2['player7_kills']+raw2['player8_kills']+raw2['player9_kills']+raw2['player10_kills']
    raw2['t2_deaths']=raw2['player6_deaths']+raw2['player7_deaths']+raw2['player8_deaths']+raw2['player9_deaths']+raw2['player10_deaths']
    raw2['t2_assists']=raw2['player6_assists']+raw2['player7_assists']+raw2['player8_assists']+raw2['player9_assists']+raw2['player10_assists']
    
    KDA=pd.concat([raw1[['t1_kills','t1_deaths','t1_assists']],raw2[['t2_kills','t2_deaths','t2_assists']],win],axis=1)
    
    KDA['t1_KDA']=(KDA['t1_kills']+KDA['t1_assists'])/KDA['t1_deaths']
    KDA['t2_KDA']=(KDA['t2_kills']+KDA['t2_assists'])/KDA['t2_deaths']
    #train['game_duration'] = round(train['gameDuration'] / 60) 
    
    x1 = KDA[(KDA['team1_win'] == 0)& (KDA['t2_KDA']<15)]['t2_KDA']
    x2 = KDA[(KDA['team1_win'] == 1)& (KDA['t1_KDA']<15)]['t1_KDA'] 
    trace0 = go.Histogram(x=x1, bingroup=25, name='team1', opacity=0.9)
    trace1 = go.Histogram(x=x2, bingroup=25, name='team2', opacity=0.9)  
    layout = go.Layout(title='胜利队伍KDA的分布') 
    data = [trace0, trace1] 
    fig = go.Figure(data, layout)
    chart = '胜利队伍KDA的分布.html'
    py.offline.plot(fig, filename=chart)
    #print(KDA.head())
    x1 = KDA[(KDA['team1_win'] == 1)&(KDA['t2_KDA']<5)&(KDA['t2_KDA']>0.05)]['t2_KDA']
    x2 = KDA[(KDA['team1_win'] == 0)&(KDA['t1_KDA']<5)&(KDA['t1_KDA']>0.05)]['t1_KDA'] 
    trace0 = go.Histogram(x=x1, bingroup=25, name='team1', opacity=0.9)
    trace1 = go.Histogram(x=x2, bingroup=25, name='team2', opacity=0.9)  
    layout = go.Layout(title='失败队伍KDA的分布') 
    data = [trace0, trace1] 
    fig = go.Figure(data, layout)
    chart = '失败队伍KDA的分布.html'
    py.offline.plot(fig, filename=chart)
    #KDA[(KDA['team1_win'] == 0)&(KDA['t2_KDA']>0)&(KDA['t2_KDA']<0.225)].info()
    
    
def plot_diff_distribute(gold): 
    #train['game_duration'] = round(train['gameDuration'] / 60) 
    x1 = gold[gold['team1_win'] == 1]['diff']
    x2 = -gold[gold['team1_win'] == 0]['diff'] 
    
    trace0 = go.Histogram(x=x1, bingroup=25, name='team1', opacity=0.9)
    trace1 = go.Histogram(x=x2, bingroup=25, name='team2', opacity=0.9)  

    layout = go.Layout(title='胜利队伍领先经济的分布') 

    data = [trace0, trace1] 
    fig = go.Figure(data, layout)
    chart = '胜利队伍领先经济的分布.html'
    py.offline.plot(fig, filename=chart)
    
def blackswan_plot(gold):
    for i in list(gold.index):
        if( gold.loc[i,'team1_win']==1 and gold.loc[i,'diff']<0 ) or (gold.loc[i,'team1_win']==0 and gold.loc[i,'diff']>0):
            gold.loc[i,'blackswan']= 1
        else :
            gold.loc[i,'blackswan']= 0

    trace0 = go.Pie(labels=gold['blackswan'].value_counts().index, 
            values=gold['blackswan'].value_counts().values,
            hole=0.5,
            marker=dict(line=dict(color='purple'))
           ) 
    layout = go.Layout(title='逆风翻盘局统计')
    data = [trace0]
    fig = go.Figure(data, layout)
    py.offline.plot(fig,filename='逆风翻盘局统计.html')
    

def transfer(win):
    if win==1:
        return 0
    else:
        return 1

def TopPick(df):
    p1=df[['team1_win','player1_championId']].rename(columns={'team1_win':'win','player1_championId':'championid'})
    p2=df[['team1_win','player2_championId']].rename(columns={'team1_win':'win','player2_championId':'championid'})
    p3=df[['team1_win','player3_championId']].rename(columns={'team1_win':'win','player3_championId':'championid'})
    p4=df[['team1_win','player4_championId']].rename(columns={'team1_win':'win','player4_championId':'championid'})
    p5=df[['team1_win','player5_championId']].rename(columns={'team1_win':'win','player5_championId':'championid'})
    p6=df[['team1_win','player6_championId']].rename(columns={'team1_win':'win','player6_championId':'championid'})
    p6["win"]=p6.apply(lambda x:transfer(x['win']),axis=1)
    p7=df[['team1_win','player7_championId']].rename(columns={'team1_win':'win','player7_championId':'championid'})
    p7["win"]=p7.apply(lambda x:transfer(x['win']),axis=1)
    p8=df[['team1_win','player8_championId']].rename(columns={'team1_win':'win','player8_championId':'championid'})
    p8["win"]=p8.apply(lambda x:transfer(x['win']),axis=1)
    p9=df[['team1_win','player9_championId']].rename(columns={'team1_win':'win','player9_championId':'championid'})
    p9["win"]=p9.apply(lambda x:transfer(x['win']),axis=1)
    p10=df[['team1_win','player10_championId']].rename(columns={'team1_win':'win','player10_championId':'championid'})
    p10["win"]=p10.apply(lambda x:transfer(x['win']),axis=1)
    t=pd.concat([p1,p2,p3,p4,p5,p6,p7,p8,p9,p10],axis=0)
    f,ax1 = plt.subplots(figsize=(15,15))
    sort_val=t.championid.value_counts().sort_values(ascending=False)    #倒序排列
    ax1=sort_val.head(10).plot(kind='bar') #取出头十个
    total_records=len(t['win'])/10
    for p in ax1.patches:
        height = p.get_height()
        ax1.text(p.get_x()+p.get_width()/2.,
                height +3,
                '{:.2f}%'.format(height/total_records*100),
                ha="center",rotation=0) #标注百分比

    plt.xticks(rotation = 45) #坐标刻度旋
    plt.yticks([total_records*0.1,total_records*0.2,total_records*0.3,total_records*0.4,total_records*0.5])
    plt.title('Pick Top 10')
    plt.show()
    return t

def winpick(t):
    w=t[t['win']==1]
    f,ax1 = plt.subplots(figsize=(15,15))
    sort_val=w.championid.value_counts().sort_values(ascending=False)    #倒序排列
    ax1=sort_val.head(10).plot(kind='bar') #取出头十个
    total_records=len(w['win'])/10
    for p in ax1.patches:
        height = p.get_height()
        ax1.text(p.get_x()+p.get_width()/2.,
                height +3,
                '{:.2f}%'.format(height/total_records*100),
                ha="center",rotation=0) #标注百分比

    plt.xticks(rotation = 45) #坐标刻度旋
    plt.yticks([total_records*0.1,total_records*0.2,total_records*0.3,total_records*0.4,total_records*0.5])
    plt.title('Pick Top 10 of Win Team')
    plt.show()
    
def wr(t):
    k=t[t['win']==1] 
    total=t.groupby('championid', as_index=False)['win'].count().rename(columns={'win':'total'})
    win=k.groupby('championid', as_index=False)['win'].count()
    Data=pd.concat([total,win],axis=1)
    Data['winrate']=Data['win']/Data['total']*100
    Data['standard']=Data['winrate']-50
    status=Data.sort_values(by='winrate', ascending = False).iloc[:,[1,2,3,4,5]]
    status['championid']=status['championid'].astype(str)
    status.head()
    f,ax = plt.subplots(figsize=(10,25))
    sns.barplot(y='championid',x='standard',data=status.sort_values(by='winrate',ascending=False))
    plt.title('Win Rate Descending')
    plt.show()

#一些设置

plt.style.use('seaborn')
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 100)
df=pd.read_csv('exp3.csv')

In [163]:
gold=team_gold() #将总经济和经济差汇总到gold表中
blackswan_plot(gold)#统计“翻盘局”的对局次数
team1_win()  #统计team1的胜率

In [183]:
team_KDA(df) #统计KDA分布

In [ ]:
plot_bar_horizontal(col_name='team1_firstBlood', target='team1_win', title_name='获得一血对获胜的影响')#获得一血对胜率影响
plot_bar_horizontal(col_name='team1_firstDragon', target='team1_win', title_name='第一条大龙对获胜的影响')#第一条大龙对获胜的影响
plot_bar_horizontal(col_name='team1_firstTower', target='team1_win', title_name='摧毁一塔对获胜的影响')#摧毁一塔对获胜的影响
plot_bar_horizontal(col_name='team1_firstInhibitor', target='team1_win', title_name='摧毁第一个水晶对获胜的影响')#摧毁第一个水晶对获胜的影响
plot_bar_horizontal(col_name='team1_firstBaron', target='team1_win', title_name='杀死第一条男爵对获胜的影响')#杀死第一条男爵对获胜的影响
plot_bar_horizontal(col_name='team1_firstRiftHerald', target='team1_win', title_name='杀死第一条峡谷先锋对获胜的影响')#击杀第一条峡谷先锋对获胜的影响
plot_diff_distribute(gold) #求胜利一队领先经济的分布

In [ ]:
t=TopPick(df) #统计出场率前10的英雄ID
winpick(t)  #统计胜利队伍出场率前10的英雄ID

In [ ]:
wr(t) #统计各个英雄的胜率